In [1]:
import pandas as pd

# Data preprocessing

In [2]:
# Import imdb data
raw_folder = './raw_data/'
imdb_rating_folder = raw_folder + 'imdb_rating/'
cmu_folder = raw_folder + 'cmu/'
baby_names_folder = raw_folder + 'baby_names_national/'

rating_df = pd.read_csv(imdb_rating_folder + 'imdb_rating.tsv', sep='\t', index_col='tconst')
display(rating_df)

,averageRating,numVotes
tconst,,
tt0000001,5.7,2004
tt0000002,5.8,269
tt0000003,6.5,1904
tt0000004,5.5,178
tt0000005,6.2,2685
...,...,...
tt9916730,7.6,11
tt9916766,7.0,22
tt9916778,7.2,36


In [3]:
# Verify the indexes are unique
print(f"Is the indexing unique ? {rating_df.index.is_unique}")

Is the indexing unique ? True


# Importing Character MetaData

In [4]:
# Import character metadata
character_df = pd.read_csv(cmu_folder + 'character.metadata.tsv', sep='\t', header=None)

# Add column names deduced from README
character_df.columns = ['wiki_ID', 'free_ID', 'release', 'char_name', 'DOB', 'gender', 'height', 'ethnicity', 'act_name', 'age_at_release', 'free_char_map1', 'free_char_map2', 'free_char_map3']

display(character_df)

,wiki_ID,free_ID,release,char_name,DOB,gender,height,ethnicity,act_name,age_at_release,free_char_map1,free_char_map2,free_char_map3
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
450664,913762,/m/03pcrp,1992-05-21,Elensh,1970-05,F,NaN,NaN,Dorothy Elias-Fahn,NaN,/m/0kr406c,/m/0kr406h,/m/0b_vcv
450665,913762,/m/03pcrp,1992-05-21,Hibiki,1965-04-12,M,NaN,NaN,Jonathan Fahn,27.0,/m/0kr405_,/m/0kr4090,/m/0bx7_j
450666,28308153,/m/0cp05t9,1957,NaN,1941-11-18,M,1.730,/m/02w7gg,David Hemmings,15.0,/m/0g8ngmc,NaN,/m/022g44
450667,28308153,/m/0cp05t9,1957,NaN,NaN,NaN,NaN,NaN,Roberta Paterson,NaN,/m/0g8ngmj,NaN,/m/0g8ngmm


# Importing Movie MetaData

In [5]:
# Import movie metadata
movie_df = pd.read_csv(cmu_folder + 'movie.metadata.tsv', sep='\t', header=None)

# Add column names deduced from README
movie_df.columns = ['wiki_ID', 'free_ID', 'mov_name', 'release', 'revenue', 'runtime', 'languages', 'countries', 'genres']
display(movie_df)
print(f"Is the wiki_ID unique ? {movie_df.wiki_ID.is_unique}")

,wiki_ID,free_ID,mov_name,release,revenue,runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}"
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ..."


Is the wiki_ID unique ? True


In [6]:
movie_df.dtypes

wiki_ID        int64
free_ID       object
mov_name      object
release       object
revenue      float64
runtime      float64
languages     object
countries     object
genres        object
dtype: object

In [7]:
# # Get the index of the row with the smallest value in 'Column1'
# min_index = movie_df['release'].idxmin()

# # Get the entire row using the index
# min_row = movie_df.loc[min_index]
# print(f"oldest movies in CMU dataset : {min_row}")

### Question 1 

lets check the percentage of missing  values in the revenue attribut of the movie dataframe

In [8]:
nb_revenue_missing = movie_df['revenue'].isna().sum()
total_movies = len(movie_df)
perc_missing = (nb_revenue_missing / total_movies)*100
print(f"Percentage of missing values in column 'revenue': {perc_missing:.2f}%")

Percentage of missing values in column 'revenue': 89.72%


lets get a list of character name in the movie dataframe

The character dataframe has not a unique index. It is due to the several Nan values present in the same movie. To tackles this we can drop the rows that have a NaN as name and see if it solve the issue

In [9]:
character_filtered = character_df.copy(deep=True)

# drop the rows with a NaN as the character name
character_filtered = character_filtered.dropna(subset=['char_name'])
# drop the rows with the same character name
character_filtered = character_filtered.drop_duplicates(subset=['wiki_ID','char_name'])

In [10]:
# DataFrame with only one entry containing all character names for a given film
character_per_movie_df = character_filtered.groupby('wiki_ID').apply(lambda x: pd.Series({
        'char_name': x['char_name'].values
    }))
display(character_per_movie_df)

,char_name
wiki_ID,
3217,"[S-Mart Clerk, Fake shemp, Ash Williams, Evil ..."
3746,"[J.F. Sebastian, Rick Deckard, Roy Batty, Rach..."
3837,"[Gabby Johnson, Taggart, Rev. Johnson, Mongo, ..."
3947,"[Don Vallens, Dorothy Vallens, Jeffrey Beaumon..."
4227,[Barry Lyndon]
...,...
37196243,"[Carlina White, Ann Pettway, Joy White, Carl W..."
37322106,[Major Samar]
37373877,"[Beth Patterson, Jennifer Jones]"


In [11]:
df_movie_names = character_per_movie_df[0:]

def split_names(names_list):
    result = []
    for name in names_list:
        split_name = name.split()
        result.extend(split_name)
    return result

df_movie_names['split_names'] = df_movie_names['char_name'].apply(split_names)

display(df_movie_names)

C:\Users\thomd\AppData\Local\Temp\ipykernel_26540\443696169.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movie_names['split_names'] = df_movie_names['char_name'].apply(split_names)


,char_name,split_names
wiki_ID,,
3217,"[S-Mart Clerk, Fake shemp, Ash Williams, Evil ...","[S-Mart, Clerk, Fake, shemp, Ash, Williams, Ev..."
3746,"[J.F. Sebastian, Rick Deckard, Roy Batty, Rach...","[J.F., Sebastian, Rick, Deckard, Roy, Batty, R..."
3837,"[Gabby Johnson, Taggart, Rev. Johnson, Mongo, ...","[Gabby, Johnson, Taggart, Rev., Johnson, Mongo..."
3947,"[Don Vallens, Dorothy Vallens, Jeffrey Beaumon...","[Don, Vallens, Dorothy, Vallens, Jeffrey, Beau..."
4227,[Barry Lyndon],"[Barry, Lyndon]"
...,...,...
37196243,"[Carlina White, Ann Pettway, Joy White, Carl W...","[Carlina, White, Ann, Pettway, Joy, White, Car..."
37322106,[Major Samar],"[Major, Samar]"
37373877,"[Beth Patterson, Jennifer Jones]","[Beth, Patterson, Jennifer, Jones]"


Given name dataset

In [12]:
# import name dataset
import os

# Define the path to your dataset folder
folder_path = 'raw_data/baby_names_national/'

# Create an empty list to store individual DataFrames
data_frames = []

# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    if filename.startswith('yob') and filename.endswith('.txt'):
        # Extract the year from the filename
        year = int(filename[3:-4])

        # Read the data from the file into a DataFrame
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, header=None, names=['name', 'gender', 'number'])

        # Add the 'year' column to the DataFrame
        df['year'] = year

        # Append the current DataFrame to the list
        data_frames.append(df)

# Concatenate all DataFrames in the list into one DataFrame
combined_data = pd.concat(data_frames, ignore_index=True)

# Set the 'name' column as the index
# combined_data.set_index('name', inplace=True)

# Display the resulting DataFrame
combined_data.head()

,name,gender,number,year
0,Mary,F,7065,1880
1,Anna,F,2604,1880
2,Emma,F,2003,1880
3,Elizabeth,F,1939,1880
4,Minnie,F,1746,1880


In [27]:
#Character names analysis

df_movie_names.head()

df_movie_names['split_names']

print(type(df_movie_names['split_names'].iloc[0]))



<class 'list'>


In [31]:
# Faire une series des nom unique des enfants 
unique_entries = combined_data['name'].unique()

# Convert the unique entries to a list
unique_entries_list = list(unique_entries)

#   display(unique_entries_list)

print(df_movie_names['split_names'].iloc[0])

intersection_set = set(df_movie_names['split_names'].iloc[0]) & set(unique_entries_list)
print(intersection_set)

bool(intersection_set)

['S-Mart', 'Clerk', 'Fake', 'shemp', 'Ash', 'Williams', 'Evil', 'Ash', 'Cowardly', 'Warrior', 'Linda', 'Sheila', 'Possessed', 'Witch', 'Duke', 'Henry', 'the', 'Red', 'Second', 'Supportive', 'Villager', 'Wiseman', 'Gold', 'Tooth', 'Blacksmith', 'Lord', 'Arthur']
{'Gold', 'Ash', 'Red', 'Linda', 'Henry', 'Warrior', 'Arthur', 'Sheila', 'Lord', 'Duke', 'Williams'}


True

In [33]:
len(df_movie_names.split_names)

Intersection_names = []

#for i in range(10) : 

#    inter = set(df_movie_names['split_names'].iloc[i]) & set(unique_entries_list)

#    Intersection_names = [Intersection_names,bool(inter)]

Intersection_names = []

for i in range(10000):
    inter = set(df_movie_names['split_names'].iloc[i]) & set(unique_entries_list)
    Intersection_names.append(bool(inter))

Intersection_names

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 Tr

In [16]:
#display(Intersection_names)

In [17]:
# df = character_per_movie_df[:10]

# def split_names(names_list):
#     split_names = [name.split() if ' ' in name else [name] for name in names_list]
#     return split_names

# df['split_names'] = df['char_name'].apply(split_names)

# display(df)

In [18]:
# df = character_per_movie_df[:10]

# def split_names(names_list):
#     result = []
#     for name in names_list:
#         if ' ' in name:
#             print(f"the name {name} has a space in it")
#             split_name = name.split()
#         else:
#             print(f"the name {name} doesn't have a space in it")
#             split_name = [name]
#         result.append(split_name)
#     return result

# df['split_names'] = df['char_name'].apply(split_names)

# display(df)

In [19]:
character_filtered = character_filtered.set_index(['wiki_ID','char_name'])
display(character_filtered)

# Verify the indexes are unique
print(f"Is the indexing unique ? {character_filtered.index.is_unique}")

free_ID     release         DOB gender  \
wiki_ID char_name                                                              
975900  Akooshay                    /m/03vyhn  2001-08-24  1958-08-26      F   
        Lieutenant Melanie Ballard  /m/03vyhn  2001-08-24  1974-08-15      F   
        Desolation Williams         /m/03vyhn  2001-08-24  1969-06-15      M   
        Sgt Jericho Butler          /m/03vyhn  2001-08-24  1967-09-12      M   
        Bashira Kincaid             /m/03vyhn  2001-08-24  1977-09-25      F   
...                                       ...         ...         ...    ...   
913762  Maj. Nexx                   /m/03pcrp  1992-05-21  1960-04-28      M   
        Lord Feff                   /m/03pcrp  1992-05-21  1960-04-28      M   
        UN Spacy Commander          /m/03pcrp  1992-05-21        1954      M   
        Silvie Gena                 /m/03pcrp  1992-05-21        1958      F   
        Elensh                      /m/03pcrp  1992-05-21     1970-05      F   

                                    height   ethnicity            act_name  \
wiki_ID char_name                                                            
975900  Akooshay                     1.620         NaN      Wanda De Jesus   
        Lieutenant Melanie Ballard   1.780  /m/044038p  Natasha Henstridge   
        Desolation Williams          1.727     /m/0x67            Ice Cube   
        Sgt Jericho Butler           1.750         NaN       Jason Statham   
        Bashira Kincaid              1.650         NaN         Clea DuVall   
...                                    ...         ...                 ...   
913762  Maj. Nexx                      NaN         NaN         Steven Blum   
        Lord Feff                      NaN         NaN         Steven Blum   
        UN Spacy Commander             NaN         NaN       Sonny Byrkett   
        Silvie Gena                    NaN         NaN       Susan Byrkett   
        Elensh                         NaN         NaN  Dorothy Elias-Fahn   

                                    age_at_release free_char_map1  \
wiki_ID char_name                                                   
975900  Akooshay                              42.0     /m/0bgchxw   
        Lieutenant Melanie Ballard            27.0      /m/0jys3m   
        Desolation Williams                   32.0      /m/0jys3g   
        Sgt Jericho Butler                    33.0     /m/02vchl6   
        Bashira Kincaid                       23.0     /m/02vbb3r   
...                                            ...            ...   
913762  Maj. Nexx                             32.0     /m/0kr40d9   
        Lord Feff                             32.0     /m/0kr408g   
        UN Spacy Commander                    38.0     /m/0kr407w   
        Silvie Gena                           34.0     /m/0kr40b9   
        Elensh                                 NaN     /m/0kr406c   

                                   free_char_map2 free_char_map3  
wiki_ID char_name                                                 
975900  Akooshay                       /m/0bgcj3x     /m/03wcfv7  
        Lieutenant Melanie Ballard     /m/0bgchn4      /m/0346l4  
        Desolation Williams            /m/0bgchn_     /m/01vw26l  
        Sgt Jericho Butler             /m/0bgchnq      /m/034hyc  
        Bashira Kincaid                /m/0bgchp9      /m/01y9xg  
...                                           ...            ...  
913762  Maj. Nexx                      /m/0kr40df      /m/044_7j  
        Lord Feff                      /m/0kr408l      /m/044_7j  
        UN Spacy Commander             /m/0kr407_      /m/0gn4bz  
        Silvie Gena                    /m/0kr40bf      /m/0gn4nd  
        Elensh                         /m/0kr406h      /m/0b_vcv  

[189341 rows x 11 columns]

Is the indexing unique ? True


In [20]:
# character_filetered.query('wiki_ID == 975900')
# # T'es trop frais Maxou le s de la veine, le fraté, le khouya
#  (+1)

# character_filetered.loc[975900]
# character_filetered.loc[975900, 'Akooshay']